# Generation of initial matrices and summary of methods

Several methods for normalization and transformation of RNAseq will be tested from [Johnson and Krishnan 2022](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-021-02568-9). A figure describing their analysis pipeline is [here](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-021-02568-9/figures/1).


## Filtering lowly expressed genes and lowly abundant OTUs




## Normalization methods for RNAseq and metataxonomics


| Data type      | Normalization         | Condition  |
|----------------|-----------------------|------------|
| Metataxonomics | Raw counts            | Joined D and N |
| Metataxonomics | CPM                   | Joined D and N |
| Metataxonomics | Relative abundance    | Joined D and N |
| Transcriptomics | Estimated counts     | D |
| Transcriptomics | CPM                  | D |
| Transcriptomics | TPM                  | D |
| Transcriptomics | TMM                  | D |
| Transcriptomics | RPKM                 | D |
| Transcriptomics | UQ                   | D |
| Transcriptomics | CTF                  | D |
| Transcriptomics | CUF                  | D |
| Transcriptomics | Others (UPDATE THIS) | D |
| Transcriptomics | Estimated counts     | N |
| Transcriptomics | CPM                  | N |
| Transcriptomics | TPM                  | N |
| Transcriptomics | TMM                  | N |
| Transcriptomics | RPKM                 | N |
| Transcriptomics | UQ                   | N |
| Transcriptomics | CTF                  | N |
| Transcriptomics | CUF                  | N |
| Transcriptomics | Others (UPDATE THIS) | N |


## Data transformation


| Data type      | Transformation                             | Reference |
|----------------|--------------------------------------------|-----------|
| Transcriptomics | asinh                                     | [Johnson and Krishnan 2022](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-021-02568-9) |
| Transcriptomics | Variance stabilizing transformation (VST) | DESeq2; used in [Priya et al 2022](https://www.nature.com/articles/s41564-022-01121-z) |


## Filtering low variance (expression or OTUs)


[Priya et al 2022](https://www.nature.com/articles/s41564-022-01121-z) used 25% quantile as cutoff for gene expression analysis.




# Obtaining the two RNAseq matrices with estimated counts (day and night)

File `0_kremling_expression_key.txt` from the `0_scripts` folder on the publication-associated [FigShare record](https://doi.org/10.6084/m9.figshare.5886769.v2) has association of identifiers between samples. The NCBI SRA records have associations between the unique identifiers in each dataset (RNAseq or 16S) and the associations with the SRA unique identifiers.

In [7]:
import re

kremling_expression_key = '/home/santosrac/Repositories/maize_microbiome_transcriptomics/correlations_rnaseq_metataxonomics/0_kremling_expression_key.txt'
sra_run_table_16s = '/home/santosrac/Repositories/maize_microbiome_transcriptomics/16S_wallace2018/SraRunInfo_Wallace_etal_2018.csv'
sra_run_table_rnaseq = '/home/santosrac/Repositories/maize_microbiome_transcriptomics/rnaseq_kremling2018/run_info/SraRunInfo_Kremling_etal_2018.csv'

# Dictionary for storing information about the samples (SRA identifier, day, day period, genotype) and associated runs (16S and RNAseq)
dict_wallace_kremling_2018 = {}

# Dictionary that will store association between Wallace and Kremling identifiers according to the key file
kremling_expression_key_dict = {}

# Counter to keep track of the number of samples with both 16S and RNAseq data
rnaseq_samples_with_16s = 0

# Counter to keep track of the number of samples without 16S data
no_16s = 0

# Dictionary to associate all 16S and RNAseq run identifiers with their sample identifier
run2my_sample_id = {}

# Reading the key file and creating a dictionary with the identifiers
with open(kremling_expression_key, 'r') as file:

    _ = file.readline()

    for line in file:
        fields = line.strip().split('\t')
        
        kremling_identifier = fields[0]
        wallace_identifier = fields[1]

        # Adding both identifiers (Wallace and Kremling) to the dictionary
        kremling_expression_key_dict[kremling_identifier] = wallace_identifier
        kremling_expression_key_dict[wallace_identifier] = kremling_identifier

# Reading the RNAseq SraRunInfo file and storing information about the samples (SRA identifier, day, day period, genotype)
with open(sra_run_table_rnaseq, 'r') as file:

    _ = file.readline()

    for line in file:
        fields = line.strip().split(',')
        fields2 = fields[11].split('_')
        rnaseq_run_id = fields[0]
        sample_id = fields2[1]
        rnaseq_genotype = fields2[2]
        day = ''
        match = re.search(r'\d+', sample_id)
        unmatched_parts = re.split(r'\d+', sample_id)
        day_period = unmatched_parts[0]
        if match:
            day = int(match.group())
        if sample_id.startswith('LMA') and rnaseq_genotype != '#N/A':
            dict_wallace_kremling_2018[fields[11]] = {'run_accession_16s': '',
                                    'run_accession_rnaseq': rnaseq_run_id,
                                    'day': day,
                                    'day_period': day_period,
                                    'genotype_16s': '',
                                    'genotype_rnaseq': rnaseq_genotype}

# Reading the 16S SraRunInfo file and storing information about the samples (SRA identifier, day, day period, genotype)
with open(sra_run_table_16s, 'r') as file:

    _ = file.readline()

    for line in file:
        fields = line.strip().split(',')
        fields2 = fields[11].split('.')
        metataxonomics_run_id = fields[0]
        day = int(fields2[1])
        day_period = fields2[0]
        for key, value in kremling_expression_key_dict.items():
            if value == fields[11]:
                if dict_wallace_kremling_2018[key]['day'] != day:
                    print('Big problem!')
                    print(day, dict_wallace_kremling_2018[key]['day'])
                    print(dict_wallace_kremling_2018[key])
                    print(value, fields[11], key)
                    exit(1)
                if dict_wallace_kremling_2018[key]['day_period'] != day_period:
                    # There is a single sample that has a different day period in the 16S data
                    if key == '10343927_LMAN8_CML505_CAACAG':
                        continue
                    else:
                        print(day_period, dict_wallace_kremling_2018[key]['day_period'])
                        print(dict_wallace_kremling_2018[key])
                        print(value, fields[11], key)
                        exit(1)
                dict_wallace_kremling_2018[key]['run_accession_16s'] = metataxonomics_run_id
                rnaseq_samples_with_16s+=1

# Checking for samples without 16S data
for key, value in dict_wallace_kremling_2018.items():
    if value['run_accession_16s'] == '':
        no_16s+=1

# Creating a dictionary to associate all 16S and RNAseq run identifiers with their sample identifier (sample identifier in SraRun table of RNAseq)
for key in dict_wallace_kremling_2018:
    if dict_wallace_kremling_2018[key]['run_accession_rnaseq']:
        run2my_sample_id[dict_wallace_kremling_2018[key]['run_accession_rnaseq']] = key
    if dict_wallace_kremling_2018[key]['run_accession_16s']:
        run2my_sample_id[dict_wallace_kremling_2018[key]['run_accession_16s']] = key

print(f'{rnaseq_samples_with_16s} sample pairs found.')
print(f'{no_16s} samples without 16S data.')

484 sample pairs found.
2 samples without 16S data.


The file with the estimated RNAseq counts generated with Salmon can be found in the [wiki describing analyses for IPBGG retret 2024](https://github.com/SantosRAC/maize_microbiome_transcriptomics/wiki/IPBGG-2024-retreat) (`Zma2_counts_matrix.tar.gz`).

Importing this matrix as a pandas dataframe and renaming the columns according to the sample names. It will be important for later filtering based on the day period.

In [20]:
import pandas as pd

# Importing expression data from Kremling et al. 2018 (estimated counts matrix on Maize v5 using Salmon after cleaning with TrimGalore)
# Values in the matrix are rounded to the nearest integer to avoid issues later when normalizing data
kremling_expression_v5 = pd.read_csv('/home/santosrac/Projects/UGA_RACS/Correlations_RNAseq_16S/Zma2_counts_matrix.txt', sep='\t')
kremling_expression_v5.set_index('Name', inplace=True)
kremling_expression_v5 = kremling_expression_v5.round(0).astype(int)

# Renaming columns to match the sample identifiers in the dictionary
kremling_expression_v5 = kremling_expression_v5.rename(columns=run2my_sample_id)
kremling_expression_v5.columns = [str(x) for x in kremling_expression_v5.columns]
print(f'There are {kremling_expression_v5.shape[0]} genes/transcripts and {kremling_expression_v5.shape[1]} samples in the expression matrix.')
kremling_expression_v5.head()

There are 39096 genes/transcripts and 486 samples in the expression matrix.


,10343927_LMAD26_CI21E_AAGTGG,10343264_LMAN26_CI21E_ATGAAC,10343927_LMAN8_B73_CACACT,10343264_LMAN26_B64_ACCAGT,10343262_LMAN8_B109_TGCTAT,10343262_LMAN8_B14A_CTCTCG,10343262_LMAN8_B57_CCTAAG,10343927_LMAD26_B77_TAATCG,10343262_LMAN8_B79_GCAGCC,10343927_LMAN8_CI187-2_GACGAT,...,10344826_LMAN8_I29_ACGTCT,10344823_LMAD8_IA2132_ACACGC,10343264_LMAD26_CML91_AACGCC,10344827_LMAN26_CML91_AATCCG,10344827_LMAN26_Ki21_AAGACA,10343927_LMAD26_Ki21_ACGTCT,10344826_LMAD8_E2558W_CGCAAC,10343927_LMAN8_E2558W_GAACCT,10344826_LMAD8_IDS69_CAGGAC,10343927_LMAN8_IDS69_ACATTA
Name,,,,,,,,,,,,,,,,,,,,,
Zm00001eb371370_T002,2,0,8,0,0,2,1,3,1,1,...,0,3,0,0,0,9,11,0,7,0
Zm00001eb371350_T001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zm00001eb371330_T001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zm00001eb371310_T001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zm00001eb371280_T001,3,6,2,0,11,2,11,0,3,3,...,4,1,1,0,11,37,21,9,3,4


Creating the association dictionary between RNAseq identifiers and the identifiers given to samples in the OTU table comprising merged day and night counts.

Then, creating the dataframe with separate RNAseq data for day and night samples.

In [60]:
# Dictionaries with the association between the Kremling identifiers and the merged day and night identifiers
kremling2_day_samples_dict = {}
kremling2_night_samples_dict = {}

for key in dict_wallace_kremling_2018.keys():
    fields = kremling_expression_key_dict[key].split('.')
    if fields[0] == 'LMAD':
        kremling2_day_samples_dict[key] = fields[2]+'_'+str(fields[1])
    elif fields[0] == 'LMAN':
        kremling2_night_samples_dict[key] = fields[2]+'_'+str(fields[1])

print(f'There are {len(kremling2_day_samples_dict)} day samples')
print(f'There are {len(kremling2_night_samples_dict)} night samples')

# Checking for duplicate identifiers and deleting duplicated records
duplicated_day = []
duplicated_night = []

for key in kremling2_night_samples_dict.keys():
    if kremling2_night_samples_dict[key] in [x for x in list(kremling2_night_samples_dict.values()) if list(kremling2_night_samples_dict.values()).count(x) > 1]:
        print(f'RNAseq sample {key} is associated with duplicated day identifier')
        duplicated_night.append(key)

for key in duplicated_night:
    print(f'Deleting {key} ({kremling2_night_samples_dict[key]})')
    del kremling2_night_samples_dict[key]

for key in kremling2_day_samples_dict.keys():
    if kremling2_day_samples_dict[key] in [x for x in list(kremling2_day_samples_dict.values()) if list(kremling2_day_samples_dict.values()).count(x) > 1]:
        print(f'RNAseq sample {key} is associated with duplicated day identifier')
        duplicated_day.append(key)

for key in duplicated_day:
    print(f'Deleting {key} ({kremling2_day_samples_dict[key]})')
    del kremling2_day_samples_dict[key]

# Creating a dataframe with RNAseq day samples
kremling_expression_v5_day = kremling_expression_v5.filter(items=list(kremling2_day_samples_dict.keys()))
print(f'RNAseq Day dataframe has {kremling_expression_v5_day.shape[1]} samples')
# Creating a dataframe with RNAseq night samples
kremling_expression_v5_night = kremling_expression_v5.filter(items=list(kremling2_night_samples_dict.keys()))
print(f'RNAseq Night dataframe has {kremling_expression_v5_night.shape[1]} samples')

# Renaming dataframes with RNAseq samples
kremling_expression_v5_day = kremling_expression_v5_day.rename(columns=kremling2_day_samples_dict)
kremling_expression_v5_night = kremling_expression_v5_night.rename(columns=kremling2_night_samples_dict)

There are 211 day samples
There are 275 night samples
RNAseq sample 10343264_LMAN26_B73_CCTGCT is associated with duplicated day identifier
RNAseq sample 10343262_LMAN8_B73_GCGCTG is associated with duplicated day identifier
RNAseq sample 10343927_LMAN8_B73_GCGCTG is associated with duplicated day identifier
RNAseq sample 10344827_LMAN26_B73_CCTGCT is associated with duplicated day identifier
Deleting 10343264_LMAN26_B73_CCTGCT (14A0029_26)
Deleting 10343262_LMAN8_B73_GCGCTG (14A0113_8)
Deleting 10343927_LMAN8_B73_GCGCTG (14A0113_8)
Deleting 10344827_LMAN26_B73_CCTGCT (14A0029_26)
RNAseq sample 10343927_LMAD8_CML505_AACGCC is associated with duplicated day identifier
RNAseq sample 10343927_LMAN8_CML505_CAACAG is associated with duplicated day identifier
Deleting 10343927_LMAD8_CML505_AACGCC (14A0570_8)
Deleting 10343927_LMAN8_CML505_CAACAG (14A0570_8)
RNAseq Day dataframe has 209 samples
RNAseq Night dataframe has 271 samples


Estimated counts datasets to be used in downstream analyses (RNAseq), before pairing with OTU matrix:

In [64]:
print(f'Dimensions of the Day matrix: {kremling_expression_v5_day.shape}')
print(f'Dimensions of the Night matrix: {kremling_expression_v5_night.shape}')

Dimensions of the Day matrix: (39096, 209)
Dimensions of the Night matrix: (39096, 271)


# Obtaining OTU data with merged day and night samples

OTU counts for day and night samples were merged in a separate notebook and imported. Values were rounded and stored as integer to avoid problems in future normalization/transformation methods.

In [22]:
otu_table_merged_d_n = pd.read_csv('/home/santosrac/Repositories/maize_microbiome_transcriptomics/16S_wallace2018/combine_day_night_samples/summed_day_night_otu_counts.tsv',
                                   sep='\t', index_col='OTU ID')
otu_table_merged_d_n = otu_table_merged_d_n.round(0).astype(int)

otu_table_merged_d_n.head()

,14A0247_8,14A0051_8,14A0381_26,14A0533_26,14A0281_8,14A0295_8,14A0169_26,14A0069_8,14A0497_26,14A0023_8,...,14A0345_8,14A0267_8,14A0009_8,14A0007_8,14A0093_26,14A0137_26,14A0265_8,14A0155_26,14A0167_26,14A0481_26
OTU ID,,,,,,,,,,,,,,,,,,,,,
4479944,1,2,3,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
995900,0,0,0,0,5,8,6,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1124709,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
541139,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
533625,0,1,2,0,0,0,0,0,40,1,...,0,0,0,1,0,0,0,2,0,2


# Obtaining pairs with both 16S and RNAseq data


In [67]:
otu_table_merged_day = otu_table_merged_d_n.filter(items=kremling_expression_v5_day.columns)
print(f'Dimensios of the Day OTU table: {otu_table_merged_day.shape}')
otu_table_merged_night = otu_table_merged_d_n.filter(items=kremling_expression_v5_night.columns)
print(f'Dimensios of the Night OTU table: {otu_table_merged_night.shape}')

Dimensios of the Day OTU table: (9057, 176)
Dimensios of the Night OTU table: (9057, 228)


In [68]:
kremling_expression_v5_day = kremling_expression_v5_day.filter(items=otu_table_merged_day.columns)
print(f'Dimensios of the Day expression table: {kremling_expression_v5_day.shape}')
kremling_expression_v5_night = kremling_expression_v5_night.filter(items=otu_table_merged_night.columns)
print(f'Dimensios of the Night expression table: {kremling_expression_v5_night.shape}')

Dimensios of the Day expression table: (39096, 176)
Dimensios of the Night expression table: (39096, 228)


# Exporting matrices to files

In [69]:
otu_table_merged_day.to_csv('/home/santosrac/Repositories/maize_microbiome_transcriptomics/correlations_rnaseq_metataxonomics/otu_table_merged_day.tsv', sep='\t')
otu_table_merged_night.to_csv('/home/santosrac/Repositories/maize_microbiome_transcriptomics/correlations_rnaseq_metataxonomics/otu_table_merged_night.tsv', sep='\t')
kremling_expression_v5_day.to_csv('/home/santosrac/Repositories/maize_microbiome_transcriptomics/correlations_rnaseq_metataxonomics/kremling_expression_v5_day.tsv', sep='\t')
kremling_expression_v5_night.to_csv('/home/santosrac/Repositories/maize_microbiome_transcriptomics/correlations_rnaseq_metataxonomics/kremling_expression_v5_night.tsv', sep='\t')